# Visualization of CNMF outputs

You must already have a batch DataFrame with CNMF outputs to run this nb

In [ ]:
from mesmerize_core import *
import numpy as np
import pandas as pd

**You will need `fastplotlib` installed for the visualizations**

In [ ]:
import fastplotlib as fpl
from ipywidgets import VBox, IntSlider, Layout

In [ ]:
pd.options.display.max_colwidth = 120

# Paths

In [ ]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/home/kushal/caiman_data/")

# batch path
batch_path = get_parent_raw_data_path().joinpath("mesmerize-batch/batch.pickle")

## Load batch

In [ ]:
df = load_batch(batch_path)
df

# Load outputs

CNMF pandas extensions API: https://mesmerize-core.readthedocs.io/en/latest/api/cnmf.html 

In [ ]:
# You can change this to plot the outputs for different batch items
index = 1

# get the motion corrected input movie as a memmap
cnmf_movie = df.iloc[index].caiman.get_input_movie()

# we can get the contours of the spatial components
contours, coms = df.iloc[index].cnmf.get_contours("all", swap_dim=False)

# and temporal components
temporal = df.iloc[index].cnmf.get_temporal("all")

ixs_good = df.iloc[index].cnmf.get_good_components()
ixs_bad = df.iloc[index].cnmf.get_bad_components()

# Visualize CNMF outputs. We'll create some simple and more complex plots.

### `ImageWidget` to view contours, and a simple `fastplotlib.Plot` to view temporal components

In [ ]:
# LineSlider is very new and experimental and is likely to change
# that's why it's not exposed as a top-level import
from fastplotlib.graphics.selectors import LinearSelector

In [ ]:
# for the image data and contours
iw_cnmf = fpl.ImageWidget(cnmf_movie, vmin_vmax_sliders=True, cmap="gnuplot2")

# add good contours to the plot within the widget
contours_graphic = iw_cnmf.gridplot[0,0].add_line_collection(contours, colors="cyan", name="contours")
contours_graphic[ixs_good].colors = "cyan"
contours_graphic[ixs_bad].colors = "magenta"


# temporal plot
plot_temporal = fpl.Plot()

temporal_graphic = plot_temporal.add_line_collection(temporal, colors="cyan", name="temporal")
temporal_graphic[ixs_good].colors = "cyan"
temporal_graphic[ixs_bad].colors = "magenta"

def update_movie(ev):
    ix = ev.pick_info["selected_index"][0]
    iw_cnmf.sliders["t"].value = ix 

# a vertical line that is syncronized to the image widget "t" (timepoint) slider
temporal_graphic.add_linear_selector(name="temp select")
plot_temporal["temp select"].selection.add_event_handler(update_movie)


# stack them
VBox([plot_temporal.show(), iw_cnmf.show()])

Auto-scale temporal plot, you can only do this after calling `show()`

In [ ]:
plot_temporal.auto_scale()
plot_temporal.camera.world.scale_x = 0.85

# Interactivity

In [ ]:
# don't worry about understanding this function
def euclidean(source, target, event, new_data):
    """maps click events to contour"""
    # calculate coms of line collection
    indices = np.array(event.pick_info["index"])
    
    coms = list()

    for contour in target.graphics:
        coors = contour.data()[~np.isnan(contour.data()).any(axis=1)]
        com = coors.mean(axis=0)
        coms.append(com)

    # euclidean distance to find closest index of com 
    indices = np.append(indices, [0])
    
    ix = int(np.linalg.norm((coms - indices), axis=1).argsort()[0])
    
    target._set_feature(feature="colors", new_data=new_data, indices=ix)
    
    return None

In [ ]:
# so we can view them one by one, first hide all of them
temporal_graphic[:].present = False

image_graphic = iw_cnmf.gridplot[0,0]["image_widget_managed"]

# link image to contours
image_graphic.link(
    "click",
    target=contours_graphic,
    feature="colors", 
    new_data="w", 
    callback=euclidean
)

# link contour color changes (which are triggered by the click events as defined above) to everything else

# thickness of contour
contours_graphic.link("colors", target=contours_graphic, feature="thickness", new_data=5)

# toggle temporal component when contour changes color
contours_graphic.link("colors", target=temporal_graphic, feature="present", new_data=True)
# autoscale temporal plot to the current temporal component
temporal_graphic[:].present.add_event_handler(plot_temporal.auto_scale)

Close plot if you GPU is slow

In [ ]:
plot_temporal.close()
iw_cnmf.gridplot.close()

# View the reconstructed movie, residuals, and reconstructed background

### We can get each of these as a mesmerize `LazyArray` which allows fast visualization of larger-than-RAM arrays that can be computed on the fly.

In [ ]:
# reconstructed movie, A * C
rcm = df.iloc[index].cnmf.get_rcm()
rcm

### `LazyArray` behaves similar to numpy arrays. But frames are computed only uppon indexing.

In [ ]:
rcm[100].shape

In [ ]:
rcm.shape

### Get size of the full array in gigabytes if it were entirely stored in RAM. This can be very large for real daa.

In [ ]:
rcm.nbytes_gb

### Some lazy arrays contain pre-computed min and max for the array, and other useful properties

In [ ]:
rcm.max

In [ ]:
gp = fpl.GridPlot((2, 2), controllers="sync")

for sp, img in zip(gp, [rcm.max_image, rcm.min_image, rcm.mean_image, rcm.std_image]):
    sp.add_image(img)
    
gp.show()

In [ ]:
gp.close()

# Visualize RCM, RCB and Residuals

In [ ]:
rcb = df.iloc[index].cnmf.get_rcb()
residuals = df.iloc[index].cnmf.get_residuals()

iw_cnmf_grid = fpl.ImageWidget(
    data=[cnmf_movie, rcm, rcb, residuals],
    vmin_vmax_sliders=True,
    cmap="gnuplot2",
    names=["movie", "A * C", "b * f", "residuals"]
)

for subplot in iw_cnmf_grid.gridplot:
    _contours = subplot.add_line_collection(contours, thickness=1.0, name="contours")
    _contours[ixs_good].colors = "cyan"
    _contours[ixs_bad].colors = "magenta"

iw_cnmf_grid.show()

In [ ]:
# temporarily hide bad components
for subplot in iw_cnmf_grid.gridplot:
    subplot["contours"][ixs_bad].present = False

In [ ]:
   # hide good components
for subplot in iw_cnmf_grid.gridplot:
    subplot["contours"][ixs_good].present = False

In [ ]:
# make everything un-hidden, indexing [:] means "everything"
for subplot in iw_cnmf_grid.gridplot:
    subplot["contours"][:].present = True

### Close the plot to free up the GPU if necessary 

In [ ]:
iw_cnmf_grid.gridplot.close()

# Visualize movie, rcm, and stack of temporal components

This example shows only good components, but as shown before you can also compare between good and bad components if you want. You could also use a `GridPlot` or `ImageWidget` to view the contours on top of the residuals and reconstructed background to evaluate if CNMF captured everything.

In [ ]:
# 1 row, 3 columns, sync the first 2 subplots plots
cnmf_grid_more = fpl.GridPlot((1, 3), controllers=[[0, 0, 1]], names=[["movie", "rcm", "temporal"]])

# movie and rcm, rcm is a lazy array and behaves similar to numpy arrays
movie_graphic = cnmf_grid_more["movie"].add_image(cnmf_movie[0], cmap="gnuplot2")
rcm_graphic = cnmf_grid_more["rcm"].add_image(rcm[0], cmap="gnuplot2")

# contours for good components
contours_good, coms = df.iloc[index].cnmf.get_contours("good", swap_dim=False)

# random colors for contours and temporal components
# make an RGBA array for each color
rand_colors = np.random.rand(len(contours_good), 4)  # [n_contours, RGBA]
rand_colors[:, -1] = 1 # set alpha = 1

# get temporal of only good components
temporal_good = df.iloc[index].cnmf.get_temporal("good")

# add contours to both movie and rcm subplots
contours_movie = cnmf_grid_more["movie"].add_line_collection(contours_good, colors=rand_colors)
contours_rcm = cnmf_grid_more["rcm"].add_line_collection(contours_good, colors=rand_colors)

# line stack of temporal components
temporal_stack = cnmf_grid_more["temporal"].add_line_stack(temporal_good, colors=rand_colors, thickness=3.0, separate=15)

# plot single temporal, just like before
plot_temporal_single = fpl.Plot()
temporal_graphic = plot_temporal_single.add_line_collection(temporal_good, colors=rand_colors)

# since this is a GridPlot and not an ImageWidget we need to define sliders
slider = IntSlider(min=0, max=cnmf_movie.shape[0] - 1, value=0, step=1)

# vertical line sliders
temporal_graphic.add_linear_selector(name="temp_single")
temporal_stack.add_linear_selector(name="temp_stack")

def update_slider(ev):
    ix = ev.pick_info["selected_index"][0]
    slider.value = ix

plot_temporal_single["temp_single"].selection.add_event_handler(update_slider)
cnmf_grid_more["temporal"]["temp_stack"].selection.add_event_handler(update_slider)

# function to update each frame
def update_frame(change):
    ix = change["new"]
    movie_graphic.data = cnmf_movie[ix]
    rcm_graphic.data = rcm[ix]
    
slider.observe(update_frame, "value")

@plot_temporal_single.renderer.add_event_handler("resize")
def update_slider_width(*args):
    width, h = plot_temporal_single.renderer.logical_size
    slider.layout = Layout(width=f"{width}px")
    
VBox([plot_temporal_single.show(), cnmf_grid_more.show(), slider])

### autoscaling

In [ ]:
plot_temporal_single.auto_scale()
plot_temporal_single.camera.scale_x = 0.85
cnmf_grid_more["temporal"].auto_scale()

### Interactivity just like before

In [ ]:
# so we can view them one by one, first hide all of them
temporal_graphic[:].present = False

# link image to contours
movie_graphic.link(
    "click",
    target=contours_movie,
    feature="colors", 
    new_data="w", 
    callback=euclidean  # we cam re-use it from before
)

# link image to contours
rcm_graphic.link(
    "click",
    target=contours_rcm,
    feature="colors", 
    new_data="w", 
    callback=euclidean  # we cam re-use it from before
)

# contours colors -> contor thickness
contours_movie.link("colors", target=contours_movie, feature="thickness", new_data=5)
contours_rcm.link("colors", target=contours_rcm, feature="thickness", new_data=5)

# contours_movie <-> contours_rcm
contours_rcm.link("colors", target=contours_movie, feature="colors", new_data="w", bidirectional=True)

# temporal stack events
temporal_stack.link("click", target=temporal_stack, feature="colors", new_data="w")
temporal_stack.link("colors", target=temporal_stack, feature="thickness", new_data=4)

# contours <-> temporal stack
contours_movie.link("colors", target=temporal_stack, feature="colors", new_data="w", bidirectional=True)

# temporal stack -> temporal single
temporal_stack.link("colors", target=temporal_graphic, feature="present", new_data=True)

# autoscale temporal plot to the current temporal component
temporal_graphic[:].present.add_event_handler(plot_temporal_single.auto_scale)

In [ ]:
plot_temporal_single.close()
cnmf_grid_more.close()